In [47]:
import string
import unicodedata
import glob
import os

all_letters = string.ascii_letters + ".'-"
n_letters = len(all_letters)


def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicode_to_ascii("'Ślusàrski"))

'Slusarski


In [48]:
def read_file(filename):
    lines = open(filename, encoding='utf-8').read().strip('\n').split('\n')
    return [unicode_to_ascii(line) for line in lines]

all_names_by_category = dict()

for filename in glob.glob('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    lines = read_file(filename)
    all_names_by_category[category] = lines

categories = list(all_names_by_category.keys())
print(categories)

['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']


In [49]:
import torch

def line_to_tensor(s):
    tensor = torch.zeros(len(s), 1, n_letters)
    for i, l in enumerate(s):
        index = all_letters.find(l)
        tensor[i][0][index] = 1
    return tensor

print(line_to_tensor('yi'))

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]]])


In [58]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, category_size, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2o = nn.Linear(category_size + input_size + hidden_size, output_size)
        self.i2h = nn.Linear(category_size + input_size + hidden_size, hidden_size)
        self.o2o = nn.Linear(output_size + hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, category, input, hidden):
        combined = torch.cat([category, input, hidden], dim=1)
        output = self.i2o(combined)
        hidden = self.i2h(combined)
        out_combined = torch.cat([output, hidden], dim=1)
        output = self.o2o(out_combined)
        output = self.softmax(self.dropout(output))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
        
hidden_size = 128
rnn = RNN(len(categories), n_letters, hidden_size, n_letters)

In [59]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(categories)
    line = randomChoice(all_names_by_category[category])
    return category, line

# One-hot vector for category
def categoryTensor(category):
    li = categories.index(category)
    tensor = torch.zeros(1, len(categories))
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor


In [64]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.init_hidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [65]:
n_iters = 10000

for i in range(n_iters):
    output, loss = train(*randomTrainingExample())
    print('%d iter, loss: %.4f'%(i, loss))

0 iter, loss: 3.5252
1 iter, loss: 3.3245
2 iter, loss: 3.3950
3 iter, loss: 3.5774
4 iter, loss: 3.3428
5 iter, loss: 3.7870
6 iter, loss: 3.5662
7 iter, loss: 3.5963
8 iter, loss: 3.4516
9 iter, loss: 3.7284
10 iter, loss: 3.4343
11 iter, loss: 3.2619
12 iter, loss: 3.4401
13 iter, loss: 3.6075
14 iter, loss: 3.4534
15 iter, loss: 3.1992
16 iter, loss: 3.7025
17 iter, loss: 3.4459
18 iter, loss: 3.7024
19 iter, loss: 3.6037
20 iter, loss: 3.6314
21 iter, loss: 3.5085
22 iter, loss: 3.3428
23 iter, loss: 3.6745
24 iter, loss: 3.4930
25 iter, loss: 3.8402
26 iter, loss: 3.7728
27 iter, loss: 3.4557
28 iter, loss: 3.5774
29 iter, loss: 3.0724
30 iter, loss: 3.4114
31 iter, loss: 3.3951
32 iter, loss: 3.5539
33 iter, loss: 3.5475
34 iter, loss: 3.3493
35 iter, loss: 3.6004
36 iter, loss: 3.3361
37 iter, loss: 3.4165
38 iter, loss: 3.5357
39 iter, loss: 3.8186
40 iter, loss: 3.4250
41 iter, loss: 3.2945
42 iter, loss: 3.3873
43 iter, loss: 3.5612
44 iter, loss: 3.2016
45 iter, loss: 3.428

366 iter, loss: 3.6925
367 iter, loss: 3.2228
368 iter, loss: 3.4081
369 iter, loss: 3.3646
370 iter, loss: 3.2944
371 iter, loss: 3.2748
372 iter, loss: 3.3643
373 iter, loss: 3.4301
374 iter, loss: 3.2139
375 iter, loss: 3.2395
376 iter, loss: 3.1711
377 iter, loss: 3.7036
378 iter, loss: 3.1969
379 iter, loss: 3.3447
380 iter, loss: 3.3615
381 iter, loss: 3.3439
382 iter, loss: 3.2336
383 iter, loss: 2.9486
384 iter, loss: 3.4441
385 iter, loss: 3.1114
386 iter, loss: 2.7226
387 iter, loss: 3.5819
388 iter, loss: 3.0794
389 iter, loss: 2.9824
390 iter, loss: 3.4595
391 iter, loss: 2.8098
392 iter, loss: 3.9810
393 iter, loss: 2.8474
394 iter, loss: 3.4109
395 iter, loss: 3.1055
396 iter, loss: 3.4204
397 iter, loss: 3.2752
398 iter, loss: 3.1591
399 iter, loss: 3.5127
400 iter, loss: 3.3565
401 iter, loss: 3.5034
402 iter, loss: 3.8457
403 iter, loss: 3.5946
404 iter, loss: 2.9689
405 iter, loss: 2.9322
406 iter, loss: 3.3781
407 iter, loss: 2.9948
408 iter, loss: 3.5327
409 iter, l

747 iter, loss: 2.9310
748 iter, loss: 4.1218
749 iter, loss: 2.6869
750 iter, loss: 3.1678
751 iter, loss: 2.6670
752 iter, loss: 3.0007
753 iter, loss: 3.7656
754 iter, loss: 3.3578
755 iter, loss: 2.9159
756 iter, loss: 2.9122
757 iter, loss: 2.7165
758 iter, loss: 3.2000
759 iter, loss: 2.9825
760 iter, loss: 3.0391
761 iter, loss: 2.9838
762 iter, loss: 3.2888
763 iter, loss: 2.7453
764 iter, loss: 2.7565
765 iter, loss: 3.0819
766 iter, loss: 3.0890
767 iter, loss: 2.8974
768 iter, loss: 3.4257
769 iter, loss: 2.9869
770 iter, loss: 3.1473
771 iter, loss: 2.7685
772 iter, loss: 2.9179
773 iter, loss: 3.5161
774 iter, loss: 2.9382
775 iter, loss: 2.8450
776 iter, loss: 3.8370
777 iter, loss: 2.8687
778 iter, loss: 3.0989
779 iter, loss: 3.4652
780 iter, loss: 3.3368
781 iter, loss: 3.0596
782 iter, loss: 3.3458
783 iter, loss: 3.0534
784 iter, loss: 3.0948
785 iter, loss: 2.7221
786 iter, loss: 2.9042
787 iter, loss: 3.0048
788 iter, loss: 2.9997
789 iter, loss: 3.0221
790 iter, l

1108 iter, loss: 3.0528
1109 iter, loss: 3.1339
1110 iter, loss: 2.7725
1111 iter, loss: 2.9225
1112 iter, loss: 2.5638
1113 iter, loss: 2.9673
1114 iter, loss: 3.5092
1115 iter, loss: 3.8521
1116 iter, loss: 3.1270
1117 iter, loss: 3.1464
1118 iter, loss: 2.6959
1119 iter, loss: 3.0186
1120 iter, loss: 2.8663
1121 iter, loss: 3.4601
1122 iter, loss: 3.6604
1123 iter, loss: 3.4609
1124 iter, loss: 2.9030
1125 iter, loss: 2.6704
1126 iter, loss: 3.0236
1127 iter, loss: 3.0113
1128 iter, loss: 3.1204
1129 iter, loss: 2.6725
1130 iter, loss: 2.9346
1131 iter, loss: 3.4772
1132 iter, loss: 2.6628
1133 iter, loss: 3.7169
1134 iter, loss: 3.2695
1135 iter, loss: 3.1461
1136 iter, loss: 3.0417
1137 iter, loss: 2.9593
1138 iter, loss: 2.7355
1139 iter, loss: 3.2529
1140 iter, loss: 2.8887
1141 iter, loss: 3.2196
1142 iter, loss: 2.9827
1143 iter, loss: 2.7150
1144 iter, loss: 2.9182
1145 iter, loss: 4.2837
1146 iter, loss: 3.7864
1147 iter, loss: 2.9136
1148 iter, loss: 3.0276
1149 iter, loss:

1458 iter, loss: 2.8747
1459 iter, loss: 3.3152
1460 iter, loss: 2.8514
1461 iter, loss: 2.8077
1462 iter, loss: 3.8203
1463 iter, loss: 3.0525
1464 iter, loss: 2.8464
1465 iter, loss: 2.5648
1466 iter, loss: 3.5427
1467 iter, loss: 3.1695
1468 iter, loss: 3.2114
1469 iter, loss: 3.4424
1470 iter, loss: 3.3553
1471 iter, loss: 3.0929
1472 iter, loss: 2.7971
1473 iter, loss: 3.2426
1474 iter, loss: 2.5585
1475 iter, loss: 3.2256
1476 iter, loss: 2.8016
1477 iter, loss: 2.9502
1478 iter, loss: 3.3386
1479 iter, loss: 3.6685
1480 iter, loss: 2.8383
1481 iter, loss: 2.9894
1482 iter, loss: 2.9462
1483 iter, loss: 3.3182
1484 iter, loss: 3.4851
1485 iter, loss: 3.1592
1486 iter, loss: 2.9189
1487 iter, loss: 2.7673
1488 iter, loss: 3.7127
1489 iter, loss: 3.1051
1490 iter, loss: 4.2474
1491 iter, loss: 3.0299
1492 iter, loss: 2.7912
1493 iter, loss: 2.8440
1494 iter, loss: 3.1056
1495 iter, loss: 3.1356
1496 iter, loss: 2.7239
1497 iter, loss: 4.2264
1498 iter, loss: 2.8478
1499 iter, loss:

...

8572 iter, loss: 2.6548
8573 iter, loss: 3.7768
8574 iter, loss: 2.8610
8575 iter, loss: 2.9718
8576 iter, loss: 3.1808
8577 iter, loss: 2.9884
8578 iter, loss: 2.8504
8579 iter, loss: 2.7792
8580 iter, loss: 2.5458
8581 iter, loss: 2.9168
8582 iter, loss: 3.2605
8583 iter, loss: 2.9711
8584 iter, loss: 2.9993
8585 iter, loss: 3.2899
8586 iter, loss: 1.8168
8587 iter, loss: 3.2307
8588 iter, loss: 2.9048
8589 iter, loss: 2.5114
8590 iter, loss: 3.2027
8591 iter, loss: 2.3112
8592 iter, loss: 2.7517
8593 iter, loss: 3.0774
8594 iter, loss: 2.4880
8595 iter, loss: 2.6709
8596 iter, loss: 3.2384
8597 iter, loss: 3.9564
8598 iter, loss: 2.6390
8599 iter, loss: 2.6525
8600 iter, loss: 2.5729
8601 iter, loss: 4.0793
8602 iter, loss: 3.2330
8603 iter, loss: 2.7533
8604 iter, loss: 2.6421
8605 iter, loss: 3.5172
8606 iter, loss: 2.6605
8607 iter, loss: 3.2257
8608 iter, loss: 3.0822
8609 iter, loss: 2.6818
8610 iter, loss: 2.6732
8611 iter, loss: 3.0276
8612 iter, loss: 3.4476
8613 iter, loss:

8929 iter, loss: 2.7222
8930 iter, loss: 2.7750
8931 iter, loss: 2.8488
8932 iter, loss: 2.6083
8933 iter, loss: 2.2495
8934 iter, loss: 3.1909
8935 iter, loss: 3.2123
8936 iter, loss: 3.1446
8937 iter, loss: 2.1365
8938 iter, loss: 2.7060
8939 iter, loss: 3.1500
8940 iter, loss: 3.0414
8941 iter, loss: 3.3355
8942 iter, loss: 2.7752
8943 iter, loss: 3.2695
8944 iter, loss: 3.5891
8945 iter, loss: 3.3528
8946 iter, loss: 2.8556
8947 iter, loss: 1.8018
8948 iter, loss: 3.3291
8949 iter, loss: 2.6529
8950 iter, loss: 2.4755
8951 iter, loss: 2.3730
8952 iter, loss: 3.0316
8953 iter, loss: 3.3072
8954 iter, loss: 2.8552
8955 iter, loss: 3.0148
8956 iter, loss: 3.1451
8957 iter, loss: 2.7112
8958 iter, loss: 2.8206
8959 iter, loss: 3.1658
8960 iter, loss: 2.9385
8961 iter, loss: 2.9253
8962 iter, loss: 3.1935
8963 iter, loss: 3.8619
8964 iter, loss: 2.5212
8965 iter, loss: 2.7155
8966 iter, loss: 2.8592
8967 iter, loss: 3.2951
8968 iter, loss: 2.9220
8969 iter, loss: 2.4640
8970 iter, loss:

9282 iter, loss: 3.2530
9283 iter, loss: 2.8454
9284 iter, loss: 3.1303
9285 iter, loss: 2.6310
9286 iter, loss: 2.9923
9287 iter, loss: 2.6631
9288 iter, loss: 2.7827
9289 iter, loss: 2.7070
9290 iter, loss: 2.6894
9291 iter, loss: 2.1437
9292 iter, loss: 3.3881
9293 iter, loss: 2.9010
9294 iter, loss: 3.2155
9295 iter, loss: 2.2740
9296 iter, loss: 2.5746
9297 iter, loss: 3.0774
9298 iter, loss: 2.7316
9299 iter, loss: 2.7064
9300 iter, loss: 3.0329
9301 iter, loss: 2.9334
9302 iter, loss: 2.9091
9303 iter, loss: 2.2817
9304 iter, loss: 2.5751
9305 iter, loss: 2.5800
9306 iter, loss: 2.8188
9307 iter, loss: 3.1390
9308 iter, loss: 3.2791
9309 iter, loss: 3.4213
9310 iter, loss: 2.9873
9311 iter, loss: 2.4201
9312 iter, loss: 3.2519
9313 iter, loss: 3.2348
9314 iter, loss: 3.0082
9315 iter, loss: 2.9585
9316 iter, loss: 3.2975
9317 iter, loss: 3.4556
9318 iter, loss: 3.2873
9319 iter, loss: 2.7469
9320 iter, loss: 2.3857
9321 iter, loss: 2.8273
9322 iter, loss: 2.4675
9323 iter, loss:

9637 iter, loss: 3.1034
9638 iter, loss: 2.7313
9639 iter, loss: 3.1381
9640 iter, loss: 3.0338
9641 iter, loss: 2.9300
9642 iter, loss: 2.9380
9643 iter, loss: 2.4488
9644 iter, loss: 2.9618
9645 iter, loss: 2.7927
9646 iter, loss: 2.7679
9647 iter, loss: 2.5093
9648 iter, loss: 2.6953
9649 iter, loss: 2.5241
9650 iter, loss: 2.2686
9651 iter, loss: 2.2197
9652 iter, loss: 2.8284
9653 iter, loss: 2.7299
9654 iter, loss: 4.0528
9655 iter, loss: 2.3476
9656 iter, loss: 2.8577
9657 iter, loss: 3.2028
9658 iter, loss: 2.9981
9659 iter, loss: 2.9454
9660 iter, loss: 3.5140
9661 iter, loss: 2.3488
9662 iter, loss: 2.5195
9663 iter, loss: 3.0208
9664 iter, loss: 2.7535
9665 iter, loss: 2.5411
9666 iter, loss: 2.1552
9667 iter, loss: 3.2629
9668 iter, loss: 3.0415
9669 iter, loss: 2.4779
9670 iter, loss: 2.2897
9671 iter, loss: 3.3366
9672 iter, loss: 2.4131
9673 iter, loss: 2.7893
9674 iter, loss: 2.6646
9675 iter, loss: 2.8263
9676 iter, loss: 2.4494
9677 iter, loss: 3.0526
9678 iter, loss:

9990 iter, loss: 2.6876
9991 iter, loss: 2.2278
9992 iter, loss: 2.5738
9993 iter, loss: 3.2981
9994 iter, loss: 2.5445
9995 iter, loss: 2.7423
9996 iter, loss: 2.7296
9997 iter, loss: 3.0067
9998 iter, loss: 3.8665
9999 iter, loss: 3.1848


In [68]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.init_hidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

In [170]:
# rnn.eval()
samples("Spanish", "ABCDEFGHIGK")

Aana
Bana
Caner
Daner
Eaner
Faner
Ganer
Haner
Ianer
Ganer
Kaner
